# Initialization

Test notebook for the C-MAPPS benchmark. Test using SVM. 

First we import the necessary packages and create the global variables.

In [1]:
#print(__doc__)

import math
import custom_scores
import CMAPSAuxFunctions
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tunableModel import TunableModel

%matplotlib inline

# Create the model

In [2]:
#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Selected from the results of running DE on the old model (250->50->1)
windowSize = 30
stride = 1
constRUL = 128

dataFolder = '../CMAPSSData'

#Create and compile the models
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SVM_RFB', svr_rbf, selected_features, dataFolder, 'scikit', window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)

# Load the data

In [3]:
#Selected from the results of running DE on the old model (250->50->1)
windowSize = 30
stride = 1
constRUL = 128
dataset = 1
tModel.changeDataset(dataset)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData(printTop=True)

Loading data for dataset 1 with window_size of 30, stride of 1 and constRUL of 128. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]


IndexError: too many indices for array

# Train the regressor

In [ ]:
#To change the model parameters the model has to be get first and then reinserted in the object
model = tModel.model
model.C = 1e4
model.epsilon = 0.2
model.gamma = 0.1
model.max_iter = 10000
model.verbose = True
tModel.changeModel('ModelRUL_SVM_RFB', model, 'scikit')

#Train the model
tModel.trainModel()

[LibSVM]

# Evaluate the performance of the regressor

In [5]:
tModel.evaluateModel(["rhs"], round=2)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
file = open("svrdt1.txt", "w")
for prediction in tModel.y_pred:
    msg = '\tEngine {}, RUL {}, Rectified RUL {}, Real RUL {}\n'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1])
    file.write (msg)
    i = i+1

print(cScores)
#print("RMSE: {}".format(rmse))
# print("RMSE2: {}".format(rmse2))
# print("RHS: {}".format(rhs))
# print("Time : {} seconds".format(time))
file.write("%s\n"%cScores)
file.write("RMSE2: "+"%s\n"%rmse2)
file.write("RHS: "+"%s\n"%rhs)
file.write("Time: "+"%s\n"%time)
file.close()

scores
{'rmse': 16.490906585145645, 'loss': 0.842518258237264, 'rhs': 4.501596478112489}
RMSE2: 16.490906585145645
RHS: 4.501596478112489
Time : 94.56158758912757 seconds


In [ ]:
# #############################################################################
# Generate sample data
X = np.sort(5 * np.random.rand(40, 1), axis=0)
y = np.sin(X).ravel()

# #############################################################################
# Add noise to targets
y[::5] += 3 * (0.5 - np.random.rand(8))

# #############################################################################
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)

# #############################################################################
# Look at the results
lw = 2
plt.scatter(X, y, color='darkorange', label='data')
plt.plot(X, y_rbf, color='navy', lw=lw, label='RBF model')
plt.plot(X, y_lin, color='c', lw=lw, label='Linear model')
plt.plot(X, y_poly, color='cornflowerblue', lw=lw, label='Polynomial model')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.show()